## Instrucciones

Edita, elabora y ejecuta los comandos y responde a las preguntas en este mismo documento. Pero **¡atención!** debes descargarlo (en formato `.ipynb`) para poder entregarlo a través del aula virtual.

## Problema 1
En la carpeta `data` hay cuatro ficheros en formato FASTQ, con secuencias de cuatro muestras diferentes de cerbero común (*Canis cerberus*). El objetivo es: primero realizar un análisis de la calidad de las secuencias y después mapearlas sobre el genoma de referencia, `data/cerber.fa`. Pero los bloques de código siguientes están desordenados y contienen errores. Corrige los errores y ejecútalos en el orden correcto para poder responder las preguntas siguietes.

In [ ]:
# BLOQUE METBREWER

library('MetBrewer')
system2(command = 'cut', args = c('-f', '4,5', 'BRZ026.sam'), stdout = 'MapQualBRZ026.txt')
system2(command = 'cut', args = c('-f', '4,5', 'BRZ031.sam'), stdout = 'MapQualBRZ031.txt')
system2(command = 'cut', args = c('-f', '4,5', 'LAN048.sam'), stdout = 'MapQualLAN048.txt')
system2(command = 'cut', args = c('-f', '4,5', 'LAN098.sam'), stdout = 'MapQualLAN098.txt')
mapq.BRZ026 <- read.table('MapQualBRZ026.txt', col.names = c('pos','mapq'))
mapq.BRZ031 <- read.table('MapQualBRZ031.txt', col.names = c('pos','mapq'))
mapq.LAN048 <- read.table('MapQualLAN048.txt', col.names = c('pos','mapq')
mapq.LAN098 <- read.table('MapQualLAN098.txt' col.names = c('pos','mapq'))
colors = met.brewer(name = 'Egypt', n = 4)
plot(density(mapq.BRZ031$mapq),  lwd = 2, col = colors[1], xlab = 'Calidad del mapeo', ylab = 'Densidad', main = '')
lines(density(mapq.BRZ026$mapq), lwd = 2, col = colors[2])
lines(density(mapq.LAN048$mapq), lwd = 2, col = colors[3])
lines(density(mapq.LAN098$mapq), lwd = 2, col = colors[4])
legend(10, y=0.08, legend = c('BRZ031','BRZ026','LAN048','LAN098'), col = colors, lwd = 2)

In [ ]:
# BLOQUE BOWTIE
BRZ026.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'BRZ026.sam',
                          seq1 = 'data/BRZ026.fq'
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
BRZ026.out

BRZ031.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'BRZ031.sam',
                          seq1 = 'data/BRZ031.fa',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
BRZ031.out

LAN048.out <  bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'LAN048.sam',
                          seq1 = 'data/LAN048.fq',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
LAN048.out

LAN098.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'LAN098.sam',
                          seq1 = 'data/LAN098.fq',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
LAN098.out

In [ ]:
# BLOQUE GGPLOT2

library(ggplot2)
suppressMessages(library('ShortRead'))
ResumenCalidad <- qa('data', type = 'fastq', pattern = '*.fa')
ResumenCalidad
head(ResumenCalidad[['baseCalls']])
report(ResumenCalidad, dest = 'ResumenCalidad.html')
ggplot(data = ResumenCalidad[['readQualityScore']],
       mapping = aes(x = quality, y = density)) +
   geom_line() +
   facet_wrap(~lane)

In [ ]:
# BLOQUE RBOWTIE2

library('Rbowtie2')
dir.create('index')
bowtie2_build(references = 'data/cerber.fq',
              bt2Index   = 'index/cerbero',
              '--quiet', overwrite = TRUE)

Contesta las preguntas siguientes:

1. Indica en qué orden crees que deben ejecutarse los pasos.
2. ¿Cuántas lecturas cortas hay en cada archivo?
3. ¿Crees que las lecturas cortas son emparejadas (*paired ends*) o no?
4. ¿Qué longitudes tienen las lecturas?
2. Se sospecha que una de las tres muestras no pertenece a la misma especie que las otras. ¿Cuál dirías que és?

## Problema 2



In [ ]:
## Bloque 2. Alineamiento

suppressMessages(library('DECIPHER'))
Secuencias <- readDNAStringSet('homolegs.fasta')
Alineadas  <- AlignSeqs(Secuencias)
BrowseSeqs(Alineadas, htmlFile = 'Alineamiento.html', openURL = FALSE)

1. ¿Qué clase de objeto es `Secuencias`, en el bloque anterior?
2. ¿Qué opinión te merece el alineamiento, y por qué?
3. Por defecto, `AlignSeqs()` usa una penalización por apertura de *gaps* de entre -18 y -16, y una penalización por alargamiento del *gap* de entre -2 y -1. Comprueba y describe el efecto de invertir esos valores. Es decir, el efecto de asignar el valor -2 (por ejemplo) a la penalización por apertura de *gaps* y el valor de -16 a la penalización por elongamiento del gap.

## Problema 3

El archivo `results.txt` contiene los resultados de una consulta realizada con el paquete `hmmer`. Concretamente, se ha comprobado qué dominios de la base de datos Pfam se encuentran presente en cada una de las proteínas del *Mycobacterium virus Che8*. El código siguiente te permite cargar la tabla de resultados en la sesión de R. Aunque para contestar algunas de las preguntas siguientes basta con abrir el archivo `results.txt` con el editor de texto, o con el comando `less -S results.txt` desde la terminal Bash.

In [ ]:
## Bloque 3. Resultados de hmmer

library(rhmmer)
resultados <- read_domtblout('results.txt')
resultados
names(resultados)

1. El paquete `hmmer` incluye varios programas: *hmmbuild*, *hmmsearch*, *hmmfetch*...  ¿Cuál de esos programas crees que ha sido utilizado para generar los resultados guardados en el archivo `results.txt`?
2. A simple vista, parece que las dos primeras líneas de la tabla de resultados son idénticas. ¿Por qué se muestra dos veces que el dominio *Phage_integrase* està presente en la proteína Q855G4?
3. El número de proteínas en las que se han buscado dominios conocidos es 112. Usando el comando `length(unique(resultados$query_name))` vemos que sólo se han encontrado dominios proteicos conocidos en 57 de esas proteínas. ¿Sabrías decir cuántos dominios diferentes se han encontrado en el proteoma del *Mycobacterium virus Che8*?
4. Los resultados de la búsqueda de dominios incluyen hasta tres valores E diferentes: uno para la secuencia completa de la proteína donde se ha encontrado algún dominio, y dos valores E para el dominio encontrado. Están descritos en las páginas 47 y 48 del documento **HMMER_Userguide.pdf**. ¿Te parecen fiables todos los resultados? ¿Sabrías decir cuántos de los dominios encontrados tiene un valor E condicional (`domain_cevalue`) menor de 1?